In [1]:
import pdal
import numpy as np

In [2]:
%load_ext autotime

In [3]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 1
    }
  ]
}"""

In [4]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
pts = r.arrays[0]

In [5]:
pts

array([(333499.29, 7395564.35, 735.34, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36606547),
       (333499.23, 7395565.79, 735.41, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36609045),
       (333499.2 , 7395566.79, 735.25, 15, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36610547),
       ...,
       (333520.96, 7395571.  , 732.5 ,  8, 1, 1, 1, 0, 6,  13., 40, 6, 363044.34273991),
       (333521.09, 7395571.84, 731.83, 18, 1, 1, 1, 0, 6,  13., 40, 6, 363044.3609089 ),
       (333520.95, 7395571.91, 729.86, 18, 1, 1, 0, 0, 6,  13., 40, 6, 363044.36412489)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8')])

In [6]:
pt_edif = pts['Classification'] == 6
len(pts[pt_edif])

300792

In [7]:
pt_ground = pts['Classification'] == 2
len(pts[pt_ground])

15890

In [40]:
len(pts)

471962

In [8]:
len(pts[pt_ground | pt_edif])

316682

In [9]:
# Escolhendo um ponto aleatório no Ground

pt = np.random.choice(pts[pt_ground])

In [10]:
coord = ['X', 'Y', 'Z']

In [11]:
pt[coord]

(333389.69, 7395295.88, 724.85)

In [12]:
# Centralizando a visão em 1.5 metros acima do solo do ponto escolhido

pts_0 = np.array([pts[pt_ground | pt_edif]['X'] - pt['X'],
         pts[pt_ground | pt_edif]['Y'] - pt['Y'],
         pts[pt_ground | pt_edif]['Z'] - (pt['Z'] + 1.5)]).T

In [13]:
## TODO
# Reprojetar os pontos para a circunferência da terra

In [14]:
## Filtrando pontos acima da visão horizontal

pts_0 = pts_0[pts_0[:, 2] >= 0.0]

In [15]:
len(pts_0)

310874

In [16]:
# Calcluando Azimute, ou melhor, um angulo começando Leste 
# crescendo no sentido anti-horario e negativo para todo Y < 0

teste_az = [[0., 1],
           [1., 0.],
           [0., -1.],
           [-1., 0.]]

teste_az = np.array(teste_az)

az = np.arctan2(teste_az[:, 1], teste_az[:, 0]) * 180 / np.pi
az

array([ 90.,   0., -90., 180.])

In [17]:
# Calculando o angulo de orientação de cada ponto

orient = np.arctan2(pts_0[:, 1], pts_0[:, 0]) * 180 / np.pi
# orient

In [18]:
# Calculando o angulo de altura de cada ponto

# np.arctan2(10, 1000) * 180 / np.pi
# np.sqrt(np.square(4) + np.square(3))

inclin = np.arctan2(pts_0[:, 2], np.sqrt(np.square(pts_0[:, 0]) + np.square(pts_0[:, 1]))) * 180 / np.pi

In [19]:
# Calculando a distância até o ponto

np.sqrt(np.square(1) + np.square(1) + np.square(10))

10.099504938362077

In [20]:
# sorted(inclin, reverse=True)

In [21]:
orient

array([67.79278812, 67.91076805, 67.9899545 , ..., 64.4924521 ,
       64.53826682, 64.56760284])

In [22]:
# Separando em fases

qt_fases = 60
fases = np.arange(-180., 180., 360./qt_fases)

pt_bins = np.digitize(orient, fases)

In [23]:
np.histogram(orient, fases)

(array([ 4876,  8001,  9183,  9692, 11324, 12515, 12949, 16920, 13098,
        10766, 10623,  9439,  7431,  6342,  3941,  2185,   787,   932,
         4409,  1571,  2575,  1820,  1153,  1310,  1606,  1759,  1931,
         1120,   592,   835,  1608,  2294,  2452,  1954,  2192,  2560,
         2845,  3143,  6267,  7933,  6236,  5841,  5141,  2262,  3254,
         2641,  5287,  5694,  5278,  5163,  5387,  6797,  7251,  5170,
         9847,  4460,  5710,  4367,  4888]),
 array([-180., -174., -168., -162., -156., -150., -144., -138., -132.,
        -126., -120., -114., -108., -102.,  -96.,  -90.,  -84.,  -78.,
         -72.,  -66.,  -60.,  -54.,  -48.,  -42.,  -36.,  -30.,  -24.,
         -18.,  -12.,   -6.,    0.,    6.,   12.,   18.,   24.,   30.,
          36.,   42.,   48.,   54.,   60.,   66.,   72.,   78.,   84.,
          90.,   96.,  102.,  108.,  114.,  120.,  126.,  132.,  138.,
         144.,  150.,  156.,  162.,  168.,  174.]))

In [24]:
np.unique(pt_bins)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60])

In [25]:
np.arange(1, qt_fases + 1)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60])

In [26]:
list(map(lambda x: [x, np.nanmax(inclin[pt_bins == x])] if len(inclin[pt_bins == x]) > 0 else [x, 0.], np.arange(1, qt_fases + 1)))
# list(map(lambda x: len(orient[pt_bins == x]), fases))
# np.nanmax(inclin[pt_bins == 3]) if len(inclin[pt_bins == 3]) > 0

[[1, 87.23457482742756],
 [2, 88.28779775920128],
 [3, 85.64946501088734],
 [4, 80.82520021528272],
 [5, 77.61386609654319],
 [6, 83.59051147862816],
 [7, 83.48652530209377],
 [8, 87.3511654485775],
 [9, 88.12870881993935],
 [10, 85.5613183981472],
 [11, 86.02265215815055],
 [12, 84.75620497002537],
 [13, 83.6768889618891],
 [14, 58.06801821951705],
 [15, 58.981738649834405],
 [16, 59.15738894175871],
 [17, 58.42641040013317],
 [18, 58.26539199780908],
 [19, 57.86651896084473],
 [20, 56.724520521989824],
 [21, 44.43999411827913],
 [22, 49.22216179046435],
 [23, 49.96466336549253],
 [24, 47.65160507423045],
 [25, 45.07097533013519],
 [26, 42.87699345847303],
 [27, 45.822550257077104],
 [28, 44.37490792935809],
 [29, 46.375223248873446],
 [30, 44.54036921980339],
 [31, 45.85606266242904],
 [32, 44.417952614442775],
 [33, 44.08171971527613],
 [34, 41.66352155798598],
 [35, 39.93054657308328],
 [36, 39.42554944058596],
 [37, 40.057638291598764],
 [38, 41.06061585905323],
 [39, 42.949820531

In [27]:
# A relação entre a densidade de pontos e a distância para obstrução do céu visível
# Quanto mais próximo um ponto está do observador, maior a obstrução que ele deve promover
# Os parâmetros portanto são distância ao observador e a distância máxima entre pontos

# Os bins então vão responder não mais a um angulo, mas uma faixa de angulos

# o Angulo é dado por arctan2(dist_max_entre_pontos/2, distancia_do_observador)

# densidade em raio
raio = 1

desvio_graus = np.degrees(np.arctan2(raio/2, \
                      np.sqrt(np.square(pts_0[:, 0]) + \
                              np.square(pts_0[:, 1]) + \
                              np.square(pts_0[:, 2]))) * 2)

desvio_graus

# Depois refatorar a função de máximo ângulo da fase

array([0.19749032, 0.19660078, 0.19598865, ..., 0.18791985, 0.18742728,
       0.18744317])

In [28]:
# Refatorando o calculo das fases

# list(map(lambda x: \
#          [x, np.nanmax(inclin[pt_bins == x])] if len(inclin[pt_bins == x]) > 0 else [x, 0.], np.arange(1, qt_fases + 1)))

orient + desvio_graus
list(map(lambda x: x, fases))

len(pts_0[((orient + desvio_graus >= 6.) | (orient - desvio_graus >= 6.) ) & ((orient + desvio_graus < 6. + 6) | (orient - desvio_graus < 6. + 6) ) ])

2677

In [ ]:
# Calcular o percentual de cada fuso
# Sendo que a área do fuso é a área da esfera 4 x Pi x r x r dividido pela quantidade de fusos
# em cada fuso subtrair a quantidade de céu visível, dao pela fórmula da calota da esfera
# 2 x Pi x r x h (sendo necessário calcular h e considenrando r constante)

In [ ]:
## TODO
# Pensar em como iterar sobre as folhas!

In [38]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.6f}".format(x)})

In [39]:
# Verificando a resolução mínima
# O tamanho em ângulo do fuso e a distância ao ponto do observador são parâmetros

angulo_fase = 360. / qt_fases
angulo_fase

distancia = np.array([1, 10, 100, 500, 1000, 10000])

np.sin(np.deg2rad(angulo_fase)) * distancia

array([0.105, 1.045, 10.453, 52.264, 104.528, 1045.285])